In [29]:
import time
import glob
import sys
import os
import uuid
import numpy as np
from PIL import Image, ImageDraw
import cv2
import pandas as pd
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [37]:
nb_dir = '/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/layout-detector/prima/src'
sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

import config
from services.get_table_cells import mask_tables 

import src.utilities.app_context as app_context
app_context.init()
app_context.application_context = dict({'task_id': 'BM-DUMMY'})





In [44]:
def draw_region(image_path,coords,save_dir,color=(255,125,0), thickness=3):
    
    image = cv2.imread(image_path)
    for region in coords :
            ground = region['boundingBox']['vertices']
            pts = []
            for pt in ground:
                pts.append([int(pt['x']) ,int(pt['y'])])
            cv2.polylines(image, [np.array(pts)],True, color, thickness)
#             cv2.putText(image, str(region['class']), (pts[0][0],pts[0][1]), font,  
#            2, (0,125,255), 3, cv2.LINE_AA)

    image_path = os.path.join(save_dir ,  '{}.png'.format(uuid.uuid4()))            
    cv2.imwrite(image_path , image)


In [45]:
save_dir = '/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/document-processor/layout-detector/prima/src/notebooks/samples'

In [46]:
image_dir = '/home/dhiraj/Documents/Anuwad/testing/Word_detector/data/table'

image = glob.glob(image_dir + '/*')

In [56]:
image_path = image[3]

In [57]:
masked_image, line_tables_regions  = mask_tables(image_path, check=False)

In [58]:
draw_region(image_path,line_tables_regions,save_dir)